In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
# Indexing the Explicit_Semantic_Ranking_Dataset
s2_doc = []
f = open('Explicit_Semantic_Ranking_Dataset/s2_doc.json', 'r')
line = f.readline()
while line:
    s2_doc.append(json.loads(line))
    line = f.readline()
# Closing file
f.close()

In [4]:
df1 = pd.DataFrame.from_dict(s2_doc)

In [83]:
df1.head()

,keyPhrases,paperAbstract,numKeyReferences,title,venue,numCitedBy,numKeyCitations,docno,ana,searchDoc
0,"[Isomorphism, Duality, DUAL, Undirected Graph,...",[We provide a correspondence between the subje...,[1],[Duality Theorems for Finite Structures (Chara...,[JCT],[54],[4],6e4eddf4d6671c37537bb5d1c9623353b62e8531,"{'bodyText': {'/m/01fngy': 2, '/m/047glwn': 1,...",[Duality Theorems for Finite Structures (Chara...
1,"[Person Reidentification, China Email, Similar...",[—This paper proposes a novel approach to pers...,[4],[Deep Ranking for Person Re-identification via...,[TIP],[6],[0],8d8afef13a8f6195d3b874231e5e767cf62f3c50,"{'bodyText': {'/m/01fngy': 4, '/m/01mwgvl': 39...",[Deep Ranking for Person Re-identification via...
2,"[Natural Language Text, PML, C.F., Automatic S...",[We present a system for identifying the seman...,[1],[Automatic Labeling of Semantic Roles],[COLING],[684],[63],04a8d271bc4384dbfbb417bfb625feb01cb44666,"{'bodyText': {'/m/01fngy': 5, '/m/07849h': 1, ...","[Automatic Labeling of Semantic Roles, We pres..."
3,"[Motion SFM, Landmarkbased Approaches, Groundp...",[Ego-motion estimation for an agile single cam...,[1],[Real-Time Simultaneous Localisation and Mappi...,[ICCV],[483],[35],102e50c2c9a8e3d00de64a26759916c926fa3db6,"{'bodyText': {'/m/03mcxw7': 2, '/m/0ndjvhb': 3...",[Real-Time Simultaneous Localisation and Mappi...
4,"[Semantic Pattern, Positive Sentiment, Sentime...",[Sentiment analysis over Twitter offer organis...,[0],[Semantic Sentiment Analysis of Twitter],[SEMWEB],[57],[1],ec4a94637ecd115219869e9df8902cb7282481e0,"{'bodyText': {'/m/0n51srf': 1, '/m/0dplw_': 1,...","[Semantic Sentiment Analysis of Twitter, Senti..."


In [7]:
# df1["searchText"] = df1.paperAbstract.apply(np.array)

In [10]:
from openai.embeddings_utils import get_embedding, cosine_similarity
import openai

In [52]:
# It takes 0.361s to get the query embedding for a query longer than the average. 
import time
s = time.time()
embedding = openai.Embedding.create(
    input="Efficient Estimation of Word Representations in Vector Space",
    engine="text-search-davinci-query-001"
)["data"][0]["embedding"]
e = time.time()
print(e - s)
len(embedding)

0.3614170551300049


12288

In [51]:
# It takes 0.409s to get the abstract/doc embedding. 
import time
s = time.time()
embedding = openai.Embedding.create(
    input=df1.paperAbstract[0][0],
    engine="text-search-davinci-doc-001"
)["data"][0]["embedding"]
e = time.time()
print(e - s)
len(embedding)

0.6764779090881348


12288

In [96]:
# Get embeddings for title and abstract.
import time
title_ab_embeddings = []
total_num = len(s2_doc)
for index, row in df1.iterrows():
    if index % 40 == 0 or index == 0:
        print("{}/{}".format(index,total_num))
    embedding = openai.Embedding.create(
    input=row['searchDoc'][0],
    engine="text-search-davinci-doc-001")
    title_ab_embeddings.append(embedding)

0/8541
40/8541
80/8541
120/8541
160/8541
200/8541
240/8541
280/8541
320/8541
360/8541
400/8541
440/8541
480/8541
520/8541
560/8541
600/8541
640/8541
680/8541
720/8541
760/8541
800/8541
840/8541
880/8541
920/8541
960/8541
1000/8541
1040/8541
1080/8541
1120/8541
1160/8541
1200/8541
1240/8541
1280/8541
1320/8541
1360/8541
1400/8541
1440/8541
1480/8541
1520/8541
1560/8541
1600/8541
1640/8541
1680/8541
1720/8541
1760/8541
1800/8541
1840/8541
1880/8541
1920/8541
1960/8541
2000/8541
2040/8541
2080/8541
2120/8541
2160/8541
2200/8541
2240/8541
2280/8541
2320/8541
2360/8541
2400/8541
2440/8541
2480/8541
2520/8541
2560/8541
2600/8541
2640/8541
2680/8541
2720/8541
2760/8541
2800/8541
2840/8541
2880/8541
2920/8541
2960/8541
3000/8541
3040/8541
3080/8541
3120/8541
3160/8541
3200/8541
3240/8541
3280/8541
3320/8541
3360/8541
3400/8541
3440/8541
3480/8541
3520/8541
3560/8541
3600/8541
3640/8541
3680/8541
3720/8541
3760/8541
3800/8541
3840/8541
3880/8541
3920/8541
3960/8541
4000/8541
4040/8541
4080/8541

In [97]:
len(title_ab_embeddings)

8541

In [100]:
np.save("title_ab_embeddings_gpt3.npy",np.array(title_ab_embeddings))

In [101]:
array2 = np.load("title_ab_embeddings_gpt3.npy", allow_pickle=True)

In [113]:
query_for_embedding = np.load("s2_query.npy", allow_pickle=True)

In [121]:
query_for_embedding = query_for_embedding.item()

In [127]:
gpt3_query_embeddings = []

In [128]:
for k, v in query_for_embedding.items():
    embedding = openai.Embedding.create(
    input=v,
    engine="text-search-davinci-query-001")
    gpt3_query_embeddings.append(embedding)

In [130]:
np.save("query_embeddings_gpt3.npy",np.array(gpt3_query_embeddings))

In [132]:
array3 = np.load("query_embeddings_gpt3.npy", allow_pickle=True)
print(len(array3))
array3[0]

100


<OpenAIObject list at 0x7fa623345450> JSON: {
  "data": [
    {
      "embedding": [
        -0.001831131405197084,
        0.002427720697596669,
        0.0024706751573830843,
        -0.007120890077203512,
        0.005272258538752794,
        0.018887221813201904,
        -0.010996334254741669,
        -0.0008726113010197878,
        -0.0039995345287024975,
        0.009959064424037933,
        -0.00022332326625473797,
        0.015641776844859123,
        -0.003515899647027254,
        0.0026790837291628122,
        -0.005450439639389515,
        -0.0038722623139619827,
        -0.01799631491303444,
        -0.00011335196904838085,
        -0.004381351638585329,
        -0.002869992284104228,
        0.0020984034053981304,
        0.00902361236512661,
        0.003154764184728265,
        -0.0035349903628230095,
        0.004012261983007193,
        0.0026488564908504486,
        -0.0056445300579071045,
        0.002776128938421607,
        -0.014661778695881367,
        0.00930361

In [147]:
cosine_similarity(array3[1]["data"][0]["embedding"], gpt3_query_embeddings[0]["data"][0]["embedding"])

0.7827879374072161

In [141]:
df1['doc_emb'] = title_ab_embeddings

In [187]:
# df is the paper abstract searched against
# query is the query
# n is the number of items returned
def search(df, query, n, query_id):
    column_name = "similarity{}".format(query_id)
    df[column_name] = df.doc_emb.apply(lambda x: cosine_similarity(x["data"][0]["embedding"], query["data"][0]["embedding"]))
    result = (
        df.sort_values(column_name, ascending=False)
        .head(n).docno.tolist()
    )
#     result = (
#         df.sort_values("similarity", ascending=False)
#         .head(n)
#     )
    return result

In [188]:
search(df1, gpt3_query_embeddings[0], 30, 1)

['34414a6175d9c3c40bcea606b7c457104e973cb3',
 '3e58d9800aa31e5db89d99dcd33e5786b7837bfd',
 '27208c88f07a1ffe97760c12be08fad3ab68fee2',
 'ad8c2721ef54c9326684762db7c9fc1378e83797',
 '2f64a414430e66bfd30a4a4837af037383150f33',
 '1d7705be75f4e29210373c2b40ee5cb6e46f0007',
 '1d385cfe161d8c2f7fcc15c5a37871bce0261a9e',
 '72d32c986b47d6b880dad0c3f155fe23d2939038',
 '317deb87586baa4ee7c7b5dfc603ebed94d1da07',
 '4c0e637fecc62b689f78af4e934bb85cdc09f048',
 '1a07186bc10592f0330655519ad91652125cd907',
 '33c68d6bc73e74ea042dc5b9fe966625565c475e',
 '96c9f11fd9901f2edeaab8cf6bbff2590cea93c4',
 'fee6892cd6f8e134d4649ade31e97d468291db9d',
 '4c46347fbc272b21468efe3d9af34b4b2bad6684',
 '77e379fd57ea44638fc628623e383eccada82689',
 '03563dfaf4d2cfa397d3c12d742e9669f4e95bab',
 '0d67362a5630ec3b7562327acc278c1c996454b5',
 'aed9097d9415d9a11937d1b5b08e4149c2b4e22d',
 '227759bc318163b2f2490690b828263f3f020cfb',
 '373f76633cc1f6c7a421e31c989842021a52fca4',
 '2c03df8b48bf3fa39054345bafabfeff15bfd11d',
 '1ffb43a8

In [160]:
import math

def calculate_nDCG(result, query_id, query_docno_gain):
    ideal_results = query_docno_gain[query_id]
    ideal_DCG = 0
    rank = 1
    for k, v in ideal_results.items():
        if rank == 1:
            ideal_DCG += v
        else:
            ideal_DCG += v / math.log2(rank)
        rank += 1
    if result[0] in ideal_results:
        nDCG = ideal_results[result[0]]
    else:
        nDCG = 0
    for i in range(1, len(result)):
        if result[i] in ideal_results:
            nDCG += ideal_results[result[i]] / math.log2(i + 1)
    nDCG /= ideal_DCG        
    return nDCG

def final_result(queries, query_docno_gain, df):
    final_score = 0
    for i in range(0, 100):
        query_id = i + 1
        query = queries[i]
        result = search(df, query, 30)
        if i == 1:
            print(result)
        nDCG = calculate_nDCG(result, query_id, query_docno_gain)
        final_score += nDCG
    return final_score / 100

In [169]:
queries = np.load("s2_query.npy", allow_pickle=True)[()]
query_docno_gain = np.load("query_docno_gain.npy", allow_pickle=True)[()]
final_score = final_result(gpt3_query_embeddings, query_docno_gain, df1)

IndexError: list index out of range

In [199]:
final_score = 0
queries = gpt3_query_embeddings
start = time.time()
for i in range(0, 100):
    print(i)
    query_id = i + 1
    query = queries[i]
    result = search(df1, query, 20, query_id)
    if i == 1:
        print(result)
    nDCG = calculate_nDCG(result, query_id, query_docno_gain)
    final_score += nDCG
final_score = final_score / 100
end = time.time()
total_time = end - start

0
1
['b218069bb452dcdf7d5618cedda8743e898cb528', '6ebe1a9faf2e7b6a2e64c3f0e11251371a9ef239', '9ddf6c4542e60c73fcbb295b6b2ed4aedf4429ab', 'a6797456656d072808909a79af517e5ed4bc9ab3', '002e061065186e28b5af4e51068058dc5a078c6e', '25870392e325331c5bd1952a88cd34bb94dfb189', '50106dec3747310427f3a893ab43e57b266c015f', 'ce3ac28a82d32ffdd82039c22f79a373a4263d4b', '6263d4e24ddd104b93b2cfee5960d527365a06fb', '15eb39e8ed2c85db3e0cc9458b7f765c7f259224', 'a9b60e28ee661a2fae6e457aab0ebf57e0453045', 'ef9a192a3e4cb251035f23b8b19010f70f699dc8', 'b56e38b60aeb107ea47488d2bd4503eaa980a07d', '144aef90608def9088b33436856d49e0b9872ee8', '31b75cee98412d0934cce459aeb5cc157e9a3022', 'e669b45d4c2352846d4d09ed8b346aa4caa00051', 'c1dd01d8b97bb653ebcc717136ff42ee5a941e2e', '2d713e202d1afacd6a58a545160e7e3f5746d69e', '608f7f9fc83ef2ad1544db2ec6b22996ee866a9c', 'ca5cb323d0499052e8eb39af2c1c0e2d55b70124']
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [200]:
final_score

0.674666535357706

In [201]:
total_time

2728.8621571063995

In [195]:
df1

,keyPhrases,paperAbstract,numKeyReferences,title,venue,numCitedBy,numKeyCitations,docno,ana,doc_emb,...,similarity91,similarity92,similarity93,similarity94,similarity95,similarity96,similarity97,similarity98,similarity99,similarity100
0,"[Isomorphism, Duality, DUAL, Undirected Graph,...",[We provide a correspondence between the subje...,[1],[Duality Theorems for Finite Structures (Chara...,[JCT],[54],[4],6e4eddf4d6671c37537bb5d1c9623353b62e8531,"{'bodyText': {'/m/01fngy': 2, '/m/047glwn': 1,...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.193265,0.232945,0.203016,0.206715,0.200531,0.236261,0.169440,0.184758,0.216016,0.176296
1,"[Person Reidentification, China Email, Similar...",[—This paper proposes a novel approach to pers...,[4],[Deep Ranking for Person Re-identification via...,[TIP],[6],[0],8d8afef13a8f6195d3b874231e5e767cf62f3c50,"{'bodyText': {'/m/01fngy': 4, '/m/01mwgvl': 39...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.197265,0.219744,0.188174,0.211811,0.283072,0.228953,0.188759,0.182811,0.235384,0.236691
2,"[Natural Language Text, PML, C.F., Automatic S...",[We present a system for identifying the seman...,[1],[Automatic Labeling of Semantic Roles],[COLING],[684],[63],04a8d271bc4384dbfbb417bfb625feb01cb44666,"{'bodyText': {'/m/01fngy': 5, '/m/07849h': 1, ...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.177507,0.263573,0.185141,0.205835,0.249029,0.211271,0.188116,0.200760,0.280898,0.184613
3,"[Motion SFM, Landmarkbased Approaches, Groundp...",[Ego-motion estimation for an agile single cam...,[1],[Real-Time Simultaneous Localisation and Mappi...,[ICCV],[483],[35],102e50c2c9a8e3d00de64a26759916c926fa3db6,"{'bodyText': {'/m/03mcxw7': 2, '/m/0ndjvhb': 3...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.183935,0.219355,0.192777,0.178377,0.248584,0.204061,0.199095,0.190706,0.219713,0.212503
4,"[Semantic Pattern, Positive Sentiment, Sentime...",[Sentiment analysis over Twitter offer organis...,[0],[Semantic Sentiment Analysis of Twitter],[SEMWEB],[57],[1],ec4a94637ecd115219869e9df8902cb7282481e0,"{'bodyText': {'/m/0n51srf': 1, '/m/0dplw_': 1,...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.179589,0.211652,0.172924,0.188139,0.226869,0.183191,0.166982,0.168966,0.248366,0.196202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8536,"[Change Point, Extreme Value, Model Order, Lin...",[Novelty detection is concerned with identifyi...,[0],[On-line novelty detection using the Kalman fi...,[ICPR],[8],[0],3c85b091aed775b44ee462fc521a089f6376e46e,"{'bodyText': {'/m/04gg9_n': 1, '/m/07rpl4': 1,...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.208914,0.241501,0.193590,0.201092,0.250384,0.204859,0.212641,0.175863,0.238244,0.244328
8537,"[Selecting, Information Processing, Probabilit...",[An approach that is common in the machine lea...,[0],[Sensor management using an active sensing app...,[SIGPRO],[54],[2],c0ea6fff135afdc03ee9f3f60f57308f8be14848,"{'bodyText': {'/m/07pwl00': 1, '/m/01gd1c': 1,...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.250343,0.220852,0.192606,0.193182,0.253715,0.215619,0.259658,0.180779,0.228406,0.187915
8538,"[Structure Of A Bayesian Network, Statistical ...",[Algorithms for learning Bayesian networks fro...,[0],[Learning Bayesian Networks is NP-Complete],[AISTATS],[584],[38],5fb874a1c8106a5b2b2779ee8e1433149109ba00,"{'bodyText': {'/m/01fngy': 1, '/m/07rpl4': 2, ...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.186206,0.222602,0.178811,0.179691,0.295897,0.215659,0.183919,0.193677,0.216137,0.227378
8539,"[Smoothing, Bilateral, Descriptor, Test Image,...",[—Very high resolution satellite images provid...,[0],[Urban-Area and Building Detection Using SIFT ...,[TGRS],[35],[2],49a970d478146a43a9b0224ea5d881511c23c110,"{'bodyText': {'/m/01fngy': 5, '/m/0zwq9p5': 1,...","{'object': 'list', 'data': [{'object': 'embedd...",...,0.220738,0.240809,0.196895,0.187558,0.265245,0.230327

In [197]:
df2 = df1.copy()

In [198]:
df2.to_csv("gpt3_similarity_scores.csv")